In [1]:
# Import the Pandas library
import pandas as pd
import numpy as np

In [2]:
# Create a reference to the school and student files
school_path = "schools_complete.csv"
student_path = "students_complete.csv"

In [3]:
# read both of these files to create data frames
school_df = pd.read_csv(school_path)
student_df = pd.read_csv(student_path)

# we will frequently need to refer to passing rates
student_df["math_passed"] = student_df["math_score"]>=70
student_df["read_passed"] = student_df["reading_score"] >=70

display(school_df)
student_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,math_passed,read_passed
0,0,Paul Bradley,M,9th,Huang High School,66,79,True,False
1,1,Victor Smith,M,12th,Huang High School,94,61,False,True
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,False,True
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,False,False
4,4,Bonnie Ray,F,9th,Huang High School,97,84,True,True


In [4]:
# create high-level snapshot of district's key metrics

# first, let's make sure we have the values right
tot_schools = school_df["school_name"].count()

tot_students = student_df["Student ID"].count()

tot_budget = school_df["budget"].sum()

avg_mathscore = student_df["math_score"].mean()

avg_readscore = student_df["reading_score"].mean()

avg_mathpass = student_df["math_passed"].mean() * 100

avg_readpass = student_df["read_passed"].mean() * 100

avg_allpass = (avg_mathpass + avg_readpass) / 2



In [5]:
# assemble our summary data into a data frame for display
disp_tbl1_df = pd.DataFrame([(tot_schools, tot_students, tot_budget, avg_mathscore, avg_readscore, avg_mathpass, avg_readpass, avg_allpass)],
                            columns=["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"])
print("District Summary")
display(disp_tbl1_df)

District Summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [6]:
# create per-school of key metrics
groupby_school_df = student_df.groupby(["school_name"])

s1 = pd.DataFrame(groupby_school_df[("Student ID")].count())
#  although as it happens, the count of individual students for each school matches the school_df value

s2 = pd.DataFrame(groupby_school_df[["math_score","reading_score","math_passed","read_passed"]].mean())

disp_tbl2_df = pd.merge(s1,s2, on=["school_name"])
disp_tbl2_df = pd.merge(disp_tbl2_df,school_df, on=["school_name"])

In [7]:
# rename columns to more accurately reflect what they represent
disp_tbl2_df = disp_tbl2_df.rename(
    columns={"Student ID":"Total Students",
             "school_name":"School Name",
             "math_score":"Average Math Score",
             "reading_score":"Average Reading Score",
            })

# add computed columns
disp_tbl2_df['Budget Per Student'] = disp_tbl2_df.apply(lambda row: row["budget"] / row["Total Students"], axis=1)
disp_tbl2_df["Pct Passed Math"] = disp_tbl2_df.apply(lambda row: row["math_passed"] * 100, axis=1)
disp_tbl2_df["Pct Passed Reading"] = disp_tbl2_df.apply(lambda row: row["read_passed"] * 100, axis=1)
disp_tbl2_df["Overall Passing Rate"] = disp_tbl2_df.apply(lambda row: (row["Pct Passed Math"] + row["Pct Passed Reading"]) / 2, axis=1)

In [8]:
# delete extraneous columns
del disp_tbl2_df["School ID"]
del disp_tbl2_df["size"]
del disp_tbl2_df["math_passed"]
del disp_tbl2_df["read_passed"]

# re-order the columns according to spec:
xyz = ["School Name","type","Total Students","budget","Budget Per Student","Average Math Score","Average Reading Score",
       "Pct Passed Math","Pct Passed Reading","Overall Passing Rate"]
disp_tbl2_df = disp_tbl2_df[xyz]

print('School Summary')
display(disp_tbl2_df)

School Summary


,School Name,type,Total Students,budget,Budget Per Student,Average Math Score,Average Reading Score,Pct Passed Math,Pct Passed Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


In [10]:
# Present Top Performing Schools according to Passing Rate
# We have a display table already, we just need to sort it 
disp_tbl2_df = disp_tbl2_df.sort_values("Overall Passing Rate",ascending=False)
print("Top Performing Schools")
display(disp_tbl2_df.head())

Top Performing Schools


,School Name,type,Total Students,budget,Budget Per Student,Average Math Score,Average Reading Score,Pct Passed Math,Pct Passed Reading,Overall Passing Rate
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


In [11]:
# Present Bottom Performing Schools according to Passing Rate
disp_tbl2_df = disp_tbl2_df.sort_values("Overall Passing Rate")
print("Bottom Performing Schools")
display(disp_tbl2_df.head())

Bottom Performing Schools


,School Name,type,Total Students,budget,Budget Per Student,Average Math Score,Average Reading Score,Pct Passed Math,Pct Passed Reading,Overall Passing Rate
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


In [12]:
# these will be useful for both math and reading summaries

# split into grades
g9_df = student_df.loc[student_df["grade"] == "9th"]
g10_df = student_df.loc[student_df["grade"] == "10th"]
g11_df = student_df.loc[student_df["grade"] == "11th"]
g12_df = student_df.loc[student_df["grade"] == "12th"]

# group each grade by school
groupby_g9_df = g9_df.groupby(["school_name"])
groupby_g10_df = g10_df.groupby(["school_name"])
groupby_g11_df = g11_df.groupby(["school_name"])
groupby_g12_df = g12_df.groupby(["school_name"])

In [13]:
# compute average math scores for each grades
mathscore_g9_df = pd.DataFrame(groupby_g9_df["math_score"].mean())
mathscore_g10_df = pd.DataFrame(groupby_g10_df["math_score"].mean())
mathscore_g11_df = pd.DataFrame(groupby_g11_df["math_score"].mean())
mathscore_g12_df = pd.DataFrame(groupby_g12_df["math_score"].mean())

# rename column according to grade level
mathscore_g9_df = mathscore_g9_df.rename(columns={"math_score":"9th"})
mathscore_g10_df = mathscore_g10_df.rename(columns={"math_score":"10th"})
mathscore_g11_df = mathscore_g11_df.rename(columns={"math_score":"11th"})
mathscore_g12_df = mathscore_g12_df.rename(columns={"math_score":"12th"})

# merge the results
disp_tbl3_df = pd.merge(mathscore_g9_df, mathscore_g10_df, on=["school_name"])
disp_tbl3_df = pd.merge(disp_tbl3_df, mathscore_g11_df, on=["school_name"])
disp_tbl3_df = pd.merge(disp_tbl3_df, mathscore_g12_df, on=["school_name"])

print ("Average Math Scores by School")
display (disp_tbl3_df)

Average Math Scores by School


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [14]:
# we already have the grade splits and groupbys from above

# compute average reading scores for each grades
readscore_g9_df = pd.DataFrame(groupby_g9_df["reading_score"].mean())
readscore_g10_df = pd.DataFrame(groupby_g10_df["reading_score"].mean())
readscore_g11_df = pd.DataFrame(groupby_g11_df["reading_score"].mean())
readscore_g12_df = pd.DataFrame(groupby_g12_df["reading_score"].mean())

# rename column according to grade level
readscore_g9_df = readscore_g9_df.rename(columns={"reading_score":"9th"})
readscore_g10_df = readscore_g10_df.rename(columns={"reading_score":"10th"})
readscore_g11_df = readscore_g11_df.rename(columns={"reading_score":"11th"})
readscore_g12_df = readscore_g12_df.rename(columns={"reading_score":"12th"})

# merge the results
disp_tbl4_df = pd.merge(readscore_g9_df, readscore_g10_df, on=["school_name"])
disp_tbl4_df = pd.merge(disp_tbl4_df, readscore_g11_df, on=["school_name"])
disp_tbl4_df = pd.merge(disp_tbl4_df, readscore_g12_df, on=["school_name"])

print ("Average Reading Scores by School")
display (disp_tbl4_df)

Average Reading Scores by School


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [15]:
# assign categories to schools
spending_bins = [0, 600, 630, 640, 9999]
spending_labels = ["up to $600","$601-630","$631-640","$641 and up"]

school_df['Budget Per Student'] = school_df.apply(lambda row: row["budget"] / row["size"], axis=1)
school_df['Spending Ranges (Per Student)'] = pd.cut(school_df["Budget Per Student"], spending_bins, labels=spending_labels)

size_bins = [0, 1775, 3000, 9999]
size_labels = ["Small","Medium","Large"]

school_df['School Size'] = pd.cut(school_df["size"], size_bins, labels=size_labels)

student_df=pd.merge(student_df, school_df, on=["school_name"])
student_df.head(10)


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,math_passed,read_passed,School ID,type,size,budget,Budget Per Student,Spending Ranges (Per Student),School Size
0,0,Paul Bradley,M,9th,Huang High School,66,79,True,False,0,District,2917,1910635,655.0,$641 and up,Medium
1,1,Victor Smith,M,12th,Huang High School,94,61,False,True,0,District,2917,1910635,655.0,$641 and up,Medium
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,False,True,0,District,2917,1910635,655.0,$641 and up,Medium
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,False,False,0,District,2917,1910635,655.0,$641 and up,Medium
4,4,Bonnie Ray,F,9th,Huang High School,97,84,True,True,0,District,2917,1910635,655.0,$641 and up,Medium
5,5,Bryan Miranda,M,9th,Huang High School,94,94,True,True,0,District,2917,1910635,655.0,$641 and up,Medium
6,6,Sheena Carter,F,11th,Huang High School,82,80,True,True,0,District,2917,1910635,655.0,$641 and up,Medium
7,7,Nicole Baker,F,12th,Huang High School,96,69,False,True,0,District,2917,1910635,655.0,$641 and up,Medium
8,8,Michael Roth,M,10th,Huang High School,95,87,True,True,0,District,2917,1910635,655.0,$641 and up,Medium
9,9,Matthew Greene,M,10th,Huang High School,96,84,True,True,0,District,2917,1910635,655.0,$641 and up,Medium


In [16]:
# create per-spending range of key metrics
groupby_spending_df = student_df.groupby(['Spending Ranges (Per Student)'])

disp_tbl5_df = pd.DataFrame(groupby_spending_df[["math_score","reading_score","math_passed","read_passed"]].mean())

# add computed columns
disp_tbl5_df["Pct Passed Math"] = disp_tbl5_df.apply(lambda row: row["math_passed"] * 100, axis=1)
disp_tbl5_df["Pct Passed Reading"] = disp_tbl5_df.apply(lambda row: row["read_passed"] * 100, axis=1)
disp_tbl5_df["Overall Passing Rate"] = disp_tbl5_df.apply(lambda row: (row["Pct Passed Math"] + row["Pct Passed Reading"]) / 2, axis=1)

# delete columns that we don't need any more
del disp_tbl5_df["math_passed"]
del disp_tbl5_df["read_passed"]

print ("Scores by School Spending")
display(disp_tbl5_df)


Scores by School Spending


,math_score,reading_score,Pct Passed Math,Pct Passed Reading,Overall Passing Rate
Spending Ranges (Per Student),,,,,
up to $600,83.362283,83.912412,93.738467,96.506335,95.122401
$601-630,79.179989,81.976641,75.600864,86.767486,81.184175
$631-640,78.050332,81.478038,71.361995,83.653734,77.507864
$641 and up,77.058995,80.958411,66.609088,80.779963,73.694526


In [17]:
# create per-size categories of key metrics
groupby_spending_df = student_df.groupby(['School Size'])

disp_tbl6_df = pd.DataFrame(groupby_spending_df[["math_score","reading_score","math_passed","read_passed"]].mean())

# add computed columns
disp_tbl6_df["Pct Passed Math"] = disp_tbl6_df.apply(lambda row: row["math_passed"] * 100, axis=1)
disp_tbl6_df["Pct Passed Reading"] = disp_tbl6_df.apply(lambda row: row["read_passed"] * 100, axis=1)
disp_tbl6_df["Overall Passing Rate"] = disp_tbl6_df.apply(lambda row: (row["Pct Passed Math"] + row["Pct Passed Reading"]) / 2, axis=1)

# delete columns that we don't need any more
del disp_tbl6_df["math_passed"]
del disp_tbl6_df["read_passed"]

print ("Scores by School Size")
display(disp_tbl6_df)


Scores by School Size


,math_score,reading_score,Pct Passed Math,Pct Passed Reading,Overall Passing Rate
School Size,,,,,
Small,83.477211,83.834479,93.619063,96.577643,95.098353
Medium,79.472501,82.235872,77.718961,87.109858,82.414409
Large,77.070764,80.928365,66.468891,81.106091,73.787491


In [18]:
# create per-School Type of key metrics
groupby_spending_df = student_df.groupby(['type'])

disp_tbl7_df = pd.DataFrame(groupby_spending_df[["math_score","reading_score","math_passed","read_passed"]].mean())

# add computed columns
disp_tbl7_df["Pct Passed Math"] = disp_tbl7_df.apply(lambda row: row["math_passed"] * 100, axis=1)
disp_tbl7_df["Pct Passed Reading"] = disp_tbl7_df.apply(lambda row: row["read_passed"] * 100, axis=1)
disp_tbl7_df["Overall Passing Rate"] = disp_tbl7_df.apply(lambda row: (row["Pct Passed Math"] + row["Pct Passed Reading"]) / 2, axis=1)

# delete columns that we don't need any more
del disp_tbl7_df["math_passed"]
del disp_tbl7_df["read_passed"]

print ("Scores by School Type")
display(disp_tbl7_df)

Scores by School Type


,math_score,reading_score,Pct Passed Math,Pct Passed Reading,Overall Passing Rate
type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
